# BTTH08: Regularized Neural Net

TODO: Ghi họ tên và MSSV của bạn (vd, Nguyễn Văn A - 1234567)

---



## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Lưu ý: tuyệt đối không gian lận. Nếu vi phạm thì bạn sẽ bị 0 điểm cho cả phần thực hành môn học. Nên nhớ mục tiêu chính ở đây là học kiến thức.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Cell` - `Run All` để chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Cell` - `Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt file `Ex08-RegularizedNeuralNet.ipynb` (không cần nộp file dữ liệu `mnist.pkl.gz`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cPickle
import gzip
import copy
# You can also import other things ...

## 3. Hàm đọc dữ liệu

Trong bài này, bạn sẽ thử nghiệm Neural Net trên bộ dữ liệu MNIST (file `mnist.pkl.gz` đính kèm). Đây là bộ dữ liệu gồm các ảnh chữ số viết tay từ 0-9 (10 lớp); mỗi ảnh có kích thước $28\times 28$ và là ảnh grayscale. Bộ dữ liệu đã được chia sẵn làm 3 tập: tập huấn luyện gồm 50000 ảnh, tập validation gồm 10000 ảnh, và tập kiểm tra gồm 10000 ảnh.

In [10]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.plk.gz').
    
    Returns
    -------
    (X_train, Y_train, X_val, Y_val, X_test, Y_test) : tuple
        X_train : numpy array, shape (N=50000, d+1=785)
            Input vectors of the training set.
        Y_train: numpy array, shape (N=50000)
            Outputs of the training set.
        X_val : numpy array, shape (N=10000, d+1=785)
            Input vectors of the validation set.
        Y_val: numpy array, shape (N=10000)
            Outputs of the validation set.
        X_test : numpy array, shape (N=10000, d+1=785)
            Input vectors of the test set.
        Y_test: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = cPickle.load(f)
    f.close()
    
    X_train, Y_train = train_data
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    
    X_val, Y_val = val_data
    X_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))
    
    X_test, Y_test = test_data
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

## 4. Hàm lan truyền tiến qua Neural Net

Trong bài này, ta sẽ sử dụng nơ-ron sigmoid ở các tẩng ẩn, và tầng softmax là tầng xuất.

In [11]:
def sigmoid(Z):
    """
    Computes sigmoid function for each element of numpy array Z.
    """
    return 1 / (1 + np.exp(-Z))

def softmax(Z):
    """
    Computes softmax function for each row of numpy array Z.
    """
    A = np.exp(Z)
    A /= np.sum(A, axis=1, keepdims=True)
    return A

def forward_prop(X, Ws):
    """
    Forward propagates X through layers of neural nets to get the final outputs.
    
    Parameters
    ----------
    X : numpy array, shape (N, d+1)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones (corresponding to x_0).
    Ws : list of numpy arrays
        The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
        d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
        d^(l) is the number of neurons (not count the +1 neuron) of layer l.
    
    Returns
    -------
    A : numpy array, shape (N, K=10)
        The maxtrix of Neural Net's output vectors; each row is an output vector (containing each 
        class's probability given the corresponding input vector).
    """
    # TODO
    A = X
    for i in range(len(Ws)):
        Z = A.dot(Ws[i])
        if i == len(Ws) - 1:
            A = Z
        else:
            A = sigmoid(Z)
            A = np.hstack((np.ones((A.shape[0],1)), A))
    return A

## 5. Hàm huấn luyện Neural Net

In [12]:
def train_neural_net(X_train, Y_train, X_val, Y_val, layer_sizes, learning_rate, mnb_size, max_patience, 
                     l2_reg_level):
    """
    Trains Neural Net on the dataset (X_train, Y_train).
    Cost function: Mean Negative Log Likelihood + L2 regularization.
    Optimization algorithm: Stochastic Gradient Descent (SGD) with early stopping.
    
    Parameters
    ----------
    X_train : numpy array, shape (N, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the training set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_train : numpy array, shape (N,)
        The vector of outputs of the training set.
    X_val : numpy array, shape (N_val, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the validation set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_val : numpy array, shape (N_val,)
        The vector of outputs of the validation set.  
    layer_sizes : list of ints
        The list of each layer' size (not count the +1 neurons).
        E.g. layer_sizes = [784, 30, 10] means: the 1st layer (input layer) has 784 neurons,
        the 2nd layer (hidden layer) has 30 neurons, the 3rd layer (output layer) has 10 neurons.
    learning_rate : float
        Learning rate of SGD.
    mnb_size : int
        Minibatch size of SGD.
    max_patience : int
        The parameter of early stopping. You'll have a `patience` variable with initial value equal to
        `max_patience`. During the training, you'll keep track of the best MBE (Mean Binary Error) 
        on the validation set; if the MBE on the validation set at the current epoch < the current 
        best one, you'll reset `patience` to `max_patience`; otherwise, `patience` -= 1. 
        When `patience` = 0, you'll terminate SGD.
    l2_reg_level : float
        The level (the coefficient) of L2 regularization.
    
    Returns
    -------
    (best_Ws, train_errs, val_errs) : tuple
        best_Ws : list of numpy arrays
            The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
            d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
            d^(l) is the number of neurons (not count the +1 neuron) of layer l.
            It's the parameters having smallest MBE on the validation set.
        train_errs: list of floats
            List of MBEs on the training set after each epoch.
        val_errs: list of floats
            List of MBEs on the validation set after each epoch.
    
    Notes
    -----
    After each epoch, you need to print out: 
    - The MBE on the training set and validation set.
    - The value of `patience`.
    E.g., 'Epoch ..., training err ..., val err ..., patience ...'.
    
    After the training, you need to print out:
    - The best MBE on the validation set.
    - The corresponding epoch.
    - The corresponding MBE on the training set.
    E.g., "Best val err ..., at epoch ..., corresponding train err ...".
    """
    # Init weights
    np.random.seed(0) # Note: this will fix the randomization so that we'll get the same random numbers each run; 
                      # it make comparisons (e.g. between different values of `l2_reg_level`) more accurate. 
    Ws = [np.random.randn(layer_sizes[l]+1, layer_sizes[l+1]) / np.sqrt(layer_sizes[l]+1) for l in range(len(layer_sizes)-1)]
    # TODO
    best_Ws_list = []
    train_errs = []
    val_errs = []
    epoch = 0
    num = 0
    sum = 0
    patience = max_patience
    co_epoch = 0
    # TODO
    N = X_train.shape[0]
    rand_idxs = range(N)
    one_hot_Y = np.zeros((N,10))
    #one_hot_Y = np.zeros((N,10))
    for i in range(N):
        one_hot_Y[i][Y_train[i]] = 1
    while (epoch != -1):
        np.random.shuffle(rand_idxs)
        for start_idx in range(0, N, mnb_size):
            mnb_X = X_train[rand_idxs[start_idx:start_idx + mnb_size]]
            mnb_Y = one_hot_Y[rand_idxs[start_idx:start_idx + mnb_size]]
            #forward prop
            As = [mnb_X]
            A = mnb_X
            for i in range(len(Ws)):
                Z = A.dot(Ws[i])
                if i == len(Ws) - 1:
                    A = softmax(Z)
                else:
                    A = sigmoid(Z)
                    A = np.hstack((np.ones((A.shape[0],1)), A))
                As.append(A)
            #back prop
            delta = As[-1] - mnb_Y
            grad = As[-2].T.dot(delta) / mnb_size
            Ws[-1] = (1 - 2 * learning_rate * l2_reg_level) * Ws[-1] - learning_rate * grad
            for l in range(2, len(layer_sizes)):
                if (l >= 3):                
                    delta = delta[:,1:].dot(Ws[-l+1].T) * As[-l] * (1 - As[-l]) #Trường hợp nhiều hơn 3 layer, bỏ đi cột đầu 
                else:
                    delta = delta.dot(Ws[-l+1].T) * As[-l] * (1 - As[-l]) #Trường hợp từ 3 layer trở xuống
                grad = As[-l-1].T.dot(delta[:, 1:]) / mnb_size 
                Ws[-l] = (1 - 2 * learning_rate * l2_reg_level) * Ws[-l] - learning_rate * grad
        A = forward_prop(X_train, Ws)
        #res = np.square(A[: - Y_train)
        for i in range(X_train.shape[0]):
            idx = A[i].argmax(axis=0)
            if (idx != Y_train[i]):
                num = num + 1
        E_in = num*1.0/(X_train.shape[0])
        num = 0
        train_errs.append(E_in)
            #validate----------------
        B = forward_prop(X_val, Ws)
        for i in range(X_val.shape[0]):
            idx = B[i].argmax(axis=0)
            if (idx != Y_val[i]):
                num = num + 1
        E_out = num*1.0/(X_val.shape[0])
        num = 0
        if (epoch != -0):
            if (E_out < val_errs[np.argmin(val_errs)]):
                patience = max_patience
            else:
                patience = patience - 1
        epoch = epoch + 1
        print "Epoch %d MBE_val: %f MBE_train: %f Patience: %f" % (epoch, E_out, E_in, patience)
        if (patience == 0):
            epoch = -1
        val_errs.append(E_out)
        best_Ws_list.append(copy.deepcopy(Ws))
        co_epoch = epoch
    best_Ws  = best_Ws_list[np.argmin(val_errs)]
    print "MBE_val best: %f Corresponding epoch: %f Corresponding MBE_train: %f" %(val_errs[np.argmin(val_errs)], np.argmin(val_errs) + 1, train_errs[np.argmin(val_errs)]) 
    return (best_Ws, train_errs, val_errs)

## 6. Thí nghiệm

Để thấy được ảnh hưởng của hệ số `l2_reg_level`, bạn sẽ dùng các hàm đã định nghĩa ở trên như sau:

1. Đọc dữ liệu.
2. Huấn luyện Neural Net trên tập huấn luyện với `layer_sizes = [784, 30, 10]`, `learning_rate = 0.1`, `mnb_size = 10`, `max_patience = 20`, và `l2_reg_level = 0, 0.0001, 0.001`. Để dễ nhìn khi chương trình `print` ra, bạn nên dùng 3 code cell cho 3 lần gọi hàm huấn luyện (ứng với 3 giá trị của `l2_reg_level`).
3. Ở cell kế tiếp, bạn sẽ vẽ ra đồ trị có trục hoàng là số lượng epoch và trục tung là độ lỗi. Với mỗi giá trị của `l2_reg_level`, bạn sẽ vẽ ra 2 đường ứng với độ lỗi MBE trên tập huấn luyện và tập validation; như vậy, trên đồ thị sẽ có tất cả 6 đường.
4. Cho nhận xét dựa vào đồ thị kết quả.
5. Cuối cùng, bạn sẽ tính và in ra độ lỗi trên tập kiểm tra của mô hình có độ lỗi nhỏ nhất trên tập validation (trong số 3 mô hình ứng với 3 giá trị của `l2_reg_level`).

(Kết quả chạy của mình: với `l2_reg_level = 0`, độ lỗi trên tập huấn luyện và tập validation lần lượt là 1.076% và 3.480%; với `l2_reg_level = 0.0001`, độ lỗi là 2.114% và 2.910%; với `l2_reg_level = 0.001`, độ lỗi là 6.940% và 6.130%.)

In [13]:
# TODO
train_X, train_Y, X_val, Y_val, test_X, test_Y = read_mnist('mnist.pkl.gz')
layer_sizes = [784, 30, 10]
(best_Ws2, E_ins2, E_outs2) = train_neural_net(train_X, train_Y, X_val, Y_val, layer_sizes, 0.1, 10, 20, 0.0001)

Epoch 1 MBE_val: 0.070300 MBE_train: 0.074800 Patience: 20.000000
Epoch 2 MBE_val: 0.058200 MBE_train: 0.059680 Patience: 20.000000
Epoch 3 MBE_val: 0.052700 MBE_train: 0.052440 Patience: 20.000000
Epoch 4 MBE_val: 0.048800 MBE_train: 0.047560 Patience: 20.000000
Epoch 5 MBE_val: 0.048500 MBE_train: 0.044240 Patience: 20.000000
Epoch 6 MBE_val: 0.043700 MBE_train: 0.039360 Patience: 20.000000
Epoch 7 MBE_val: 0.042700 MBE_train: 0.037820 Patience: 20.000000
Epoch 8 MBE_val: 0.039900 MBE_train: 0.035960 Patience: 20.000000
Epoch 9 MBE_val: 0.040700 MBE_train: 0.035800 Patience: 19.000000
Epoch 10 MBE_val: 0.037600 MBE_train: 0.034280 Patience: 20.000000
Epoch 11 MBE_val: 0.039500 MBE_train: 0.032580 Patience: 19.000000
Epoch 12 MBE_val: 0.037100 MBE_train: 0.031120 Patience: 20.000000
Epoch 13 MBE_val: 0.038700 MBE_train: 0.030440 Patience: 19.000000
Epoch 14 MBE_val: 0.035600 MBE_train: 0.028600 Patience: 20.000000
Epoch 15 MBE_val: 0.038200 MBE_train: 0.029780 Patience: 19.000000
Epoc

In [14]:
(best_Ws1, E_ins1, E_outs1) = train_neural_net(train_X, train_Y, X_val, Y_val, layer_sizes, 0.1, 10, 20, 0)

Epoch 1 MBE_val: 0.070300 MBE_train: 0.074800 Patience: 20.000000
Epoch 2 MBE_val: 0.058200 MBE_train: 0.059680 Patience: 20.000000
Epoch 3 MBE_val: 0.052700 MBE_train: 0.052440 Patience: 20.000000
Epoch 4 MBE_val: 0.048800 MBE_train: 0.047560 Patience: 20.000000


KeyboardInterrupt: 

In [ ]:
(best_Ws3, E_ins3, E_outs3) = train_neural_net(train_X, train_Y, X_val, Y_val, layer_sizes, 0.1, 10, 20, 0.001)

In [ ]:
plt.plot(range(len(E_ins1)), E_ins1, label = 'E_train - 0')
plt.plot(range(len(E_outs1)), E_outs1, label = 'E_val - 0')
plt.plot(range(len(E_ins2)), E_ins2, label = 'E_train - 0.0001')
plt.plot(range(len(E_outs2)), E_outs2, label = 'E_val - 0.0001')
plt.plot(range(len(E_ins3)), E_ins3, label = 'E_train - 0.001')
plt.plot(range(len(E_out3)), E_outs3, label = 'E_val - 0.001')
plt.legend()
plt.show()

Với l2_reg_level quá lớn thì dẫn đến việc underfitting và quá nhỏ thì dẫn đến overfitting trên tập validate, vì thế ta phải chọn hệ số cho phù hợp.

In [ ]:
min1 = E_outs1[np.argmin(E_outs1)]
min2 = E_outs2[np.argmin(E_outs2)]
min3 = E_outs3[np.argmin(E_outs3)]
min = np.min([min1, min2, min3])
A = []
if (min == min1):
    A = forward_prop(test_X, best_Ws1)
    print "L2_reg: 0"
elif (min == min2):
    A = forward_prop(test_X, best_Ws2)
    print "L2_reg: 0.0001"
elif (min == min3):
    A = forward_prop(test_X, best_Ws3)
    print "L2_reg: 0.001"
for i in range(test_X.shape[0]):
    idx = A[i].argmax(axis=0)
    if (idx != test_Y[i]):
        num = num + 1
print "Error : %f" %(num*1.0) / test_X.shape[0]